# Canonical (dataclass) task form

## Python-task definitions

In [ ]:
from pprint import pprint
from pydra.engine.helpers import fields_dict
from pydra.engine.specs import PythonDef, PythonOutputs
from pydra.design import python


@python.define
class CanonicalPythonDef(PythonDef["CanonicalPythonDef.Outputs"]):
    """Canonical Python task definition class for testing

    Args:
        a: First input
            to be inputted
        b: Second input
    """

    a: int
    b: float = 2.0

    class Outputs(PythonOutputs):
        """
        Args:
            c: Sum of a and b
            d: Product of a and b
        """

        c: float
        d: float

    @staticmethod
    def function(a, b):
        return a + b, a * b

pprint(fields_dict(CanonicalPythonDef))
pprint(fields_dict(CanonicalPythonDef.Outputs))

## Shell-task definitions

In [ ]:
import os
from pathlib import Path
from fileformats import generic
from pydra.design import shell
from pydra.engine.specs import ShellDef, ShellOutputs
from pydra.utils.typing import MultiInputObj


@shell.define
class CpWithSize(ShellDef["CpWithSize.Outputs"]):

    executable = "cp"

    in_fs_objects: MultiInputObj[generic.FsObject]
    recursive: bool = shell.arg(argstr="-R")
    text_arg: str = shell.arg(argstr="--text-arg")
    int_arg: int | None = shell.arg(argstr="--int-arg")
    tuple_arg: tuple[int, str] | None = shell.arg(argstr="--tuple-arg")

    class Outputs(ShellOutputs):

        @staticmethod
        def get_file_size(out_file: Path) -> int:
            """Calculate the file size"""
            result = os.stat(out_file)
            return result.st_size

        out_file: generic.File
        out_file_size: int = shell.out(callable=get_file_size)


pprint(fields_dict(CpWithSize))
pprint(fields_dict(CpWithSize.Outputs))

## Workflow definitions

Like with Python and shell tasks, it is also possible to specify workflows in "dataclass form" in order to be more explicit to linters, which can be worth the extra effort when creating a suite of workflows to be shared publicly. In this case the workflow constructor should be a static method of the dataclasss named `constructor`.

This form also lends itself to defining custom converters and validators on the fields

In [ ]:
from pydra.design import python, workflow
from pydra.engine.specs import WorkflowDef, WorkflowOutputs

# Example python task definitions
@python.define
def Add(a, b):
    return a + b


@python.define
def Mul(a, b):
    return a * b


@workflow.define
class CanonicalWorkflowDef(WorkflowDef["CanonicalWorkflowDef.Outputs"]):

    @staticmethod
    def a_converter(value):
        if value is None:
            return value
        return float(value)

    a: int
    b: float = workflow.arg(
        help_string="A float input",
        converter=a_converter,
    )

    @staticmethod
    def constructor(a, b):
        add = workflow.add(Add(a=a, b=b))
        mul = workflow.add(Mul(a=add.out, b=b))
        return mul.out

    class Outputs(WorkflowOutputs):
        out: float